In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
load_dotenv(override=True)

gemini_key = os.getenv("GOOGLE_API_KEY")
if gemini_key:
    print(f"GEMINI API Key exists")
else:
    print("GEMINI API Key not set")

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL = 'gemini-2.5-flash-preview-05-20'
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_key)

In [ ]:
# initializing db
DB = "books.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE books (
        title TEXT PRIMARY KEY,
        author TEXT,
        available BOOLEAN,
        borrower TEXT
    )
    ''')


In [ ]:
def add_book(title, author):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        try:
            cursor.execute(
                'INSERT INTO books (title, author, available) VALUES (?,?,?)',
                (title, author, 1)
            )
            return f'Book "{title}" added successfully.'
        except sqlite3.IntegrityError:
            return f'Book "{title}" already exists.'

In [ ]:
# Add some dummy books
add_book("Python 101", "Michael Driscoll")
add_book("Clean Code", "Robert C. Martin")
add_book("Deep Learning 101", "Ian Goodfellow")
add_book("The Pragmatic Programmer", "Andrew Hunt")
add_book("Introduction to Algorithms", "Cormen, Leiserson, Rivest, Stein")

In [ ]:
def get_book_info(title):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'SELECT title, author, available, borrower FROM books WHERE title = ?',
            (title,)
        )
        result = cursor.fetchone()
        if not result:
            return f'Book "{title}" not found.'

        title, author, available, borrower = result
        status = 'available' if available else f'borrowed by {borrower}'        
        return f'"{title}" by {author} is {status}.'

In [ ]:
def borrow_book(title, borrower_name):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT available FROM books WHERE title = ?', (title,))
        result = cursor.fetchone()
        if not result:
            return f'Book "{title}" not found.'
        if not result[0]:
            return f'Sorry, "{title}" is already borrowed.'

        cursor.execute(
            'UPDATE books SET borrower=?, available=? WHERE title=?',
            (borrower_name, 0, title)
        )
    return f'"{title}" has been borrowed by {borrower_name}.'

In [ ]:
def return_book(title):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT available FROM books WHERE title = ?', (title,))
        result = cursor.fetchone()
        if not result:
            return f'Book "{title}" not found.'
        if result[0]:
            return f'"{title}" is already available.'

        cursor.execute(
            'UPDATE books SET borrower=?, available=? WHERE title=?',
            (None, 1, title)
        )
    return f'"{title}" has been returned and is now available.'

In [ ]:
add_book_tool = {
    "name": "add_book",
    "description": "Add a new book to the library database.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string", "description": "The book's title"},
            "author": {"type": "string", "description": "The author's name"}
        },
        "required": ["title", "author"],
        "additionalProperties": False
    }
}

In [ ]:
get_book_info_tool = {
    "name": "get_book_info",
    "description": "Get information about a book (author, availability, borrower).",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string", "description": "The book's title"}
        },
        "required": ["title"],
        "additionalProperties": False
    }
}

In [ ]:
borrow_book_tool = {
    "name": "borrow_book",
    "description": "Borrow a book from the library if it is available.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string", "description": "The book's title"},
            "borrower_name": {"type": "string", "description": "Name of the borrower"}
        },
        "required": ["title", "borrower_name"],
        "additionalProperties": False
    }
}

In [ ]:
return_book_tool = {
    "name": "return_book",
    "description": "Return a borrowed book to the library.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string", "description": "The book's title"}
        },
        "required": ["title"],
        "additionalProperties": False
    }
}

In [ ]:
system_message = """
You are a helpful library assistant. 
You can add books, provide book info, let users borrow books, and handle returns.
Always respond in a friendly, clear way suitable for a chatbot interface.
Use the database functions when appropriate to answer questions about books.
"""

tools = [
    {"type": "function", "function": add_book_tool},
    {"type": "function", "function": get_book_info_tool},
    {"type": "function", "function": borrow_book_tool},
    {"type": "function", "function": return_book_tool}
]

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "add_book":
            result = add_book(arguments.get("title"), arguments.get("author"))
        elif tool_call.function.name == "get_book_info":
            result = get_book_info(arguments.get("title"))
        elif tool_call.function.name == "borrow_book":
            result = borrow_book(arguments.get("title"), arguments.get("borrower_name"))
        elif tool_call.function.name == "return_book":
            result = return_book(arguments.get("title"))
        else:
            result = f"Unknown tool: {tool_call.function.name}"

        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = gemini.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # handle any tool calls
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        print(message)
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = gemini.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()